<a href="https://colab.research.google.com/github/Muzsek/Spam-or-Ham/blob/main/S_H_AIO_Full_Done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir data

In [3]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
# Set the path to the file you'd like to load
file_path = "spam.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "uciml/sms-spam-collection-dataset",
  file_path,
  pandas_kwargs={"encoding": "latin-1"}
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

/tmp/ipython-input-1142090527.py:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [4]:
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
if df.columns[2] == 'Unnamed: 2':
  df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, errors='ingore') # Egyszer lefut, utána hiba
else:
    print("Deleted")

for i in df.columns:
  print(i)
df.head(5)

v1
v2


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess(text):
  text = text.lower()

  text = text.translate(str.maketrans('', '', string.punctuation))

  tokens = word_tokenize(text)

  filtered_tokens = [word for word in tokens if word not in stop_words]

  return ' '.join(filtered_tokens)

df['v2_clean'] = df['v2'].apply(preprocess)
df[['v2', 'v2_clean']].head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,v2,v2_clean
0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though


In [7]:
X = df['v2_clean']
y = df['v1']

In [8]:
for i in range(5):
  print(f"{y[i]} || {X[i]}")

ham || go jurong point crazy available bugis n great world la e buffet cine got amore wat
ham || ok lar joking wif u oni
spam || free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s
ham || u dun say early hor u c already say
ham || nah dont think goes usf lives around though


In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

# y_encoded_list = []

# for i in range(5):
#   y_encoded_list.append(int(y_encoded[i]))

# print(list(zip(y[:5], y_encoded_list[:5]))) # Integer output

print(list(zip(y[:5], y_encoded[:5]))) # Np output

[('ham', np.int64(0)), ('ham', np.int64(0)), ('spam', np.int64(1)), ('ham', np.int64(0)), ('ham', np.int64(0))]


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size = 0.2,
    random_state = 42,
    stratify = y
)

In [11]:
for i in range(1,5+1):
  print(f"{i}, Train label: {y_train.iloc[i]} || Train data: {X_train.iloc[i]}")

print(f"\nAll training data: {len(X_train)}")
print(f"All testing data: {len(X_test)}")

1, Train label: ham || Train data: wont wats wit guys
2, Train label: ham || Train data: ok ksry knw 2 sivatats askd
3, Train label: ham || Train data: stand away doesnt heart ache without dont wonder dont crave
4, Train label: ham || Train data: finished work yet something
5, Train label: ham || Train data: hhahhaahahah rofl wtf nig leonardo room something

All training data: 4457
All testing data: 1115


In [12]:
import pandas as pd

train_counts = pd.Series(y_train).value_counts(normalize = True)
test_counts = pd.Series(y_train).value_counts(normalize = True)

print(f"Train counts: {train_counts}")
print(f"Test counts: {test_counts}")

Train counts: v1
ham     0.865829
spam    0.134171
Name: proportion, dtype: float64
Test counts: v1
ham     0.865829
spam    0.134171
Name: proportion, dtype: float64


### Bag of words

In [13]:
### Bag of Words preparation

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

bow_vectorizer = CountVectorizer()

X_train_vectorized = bow_vectorizer.fit_transform(X_train)
X_test_vectorized = bow_vectorizer.transform(X_test)


print("Train BoW shape:", X_train_vectorized.shape)
print("Test BoW shape:", X_test_vectorized.shape)


sample_index = 0
print("Eredeti szöveg:", X_train.iloc[sample_index])
print("BoW reprezentáció:", X_train_vectorized[sample_index].toarray())
print("Feature nevek (szavak):", bow_vectorizer.get_feature_names_out()[:20])

Train BoW shape: (4457, 8271)
Test BoW shape: (1115, 8271)
Eredeti szöveg: going nothing greatbye
BoW reprezentáció: [[0 0 0 ... 0 0 0]]
Feature nevek (szavak): ['008704050406' '0089my' '0121' '01223585236' '01223585334' '02' '020603'
 '0207' '02070836089' '02072069400' '02073162414' '02085076972' '020903'
 '021' '050703' '0578' '06' '060505' '061104' '07008009200']


In [65]:
## Naive Bayes with BoW

from sklearn.naive_bayes import MultinomialNB

bow_nb_model = MultinomialNB()
bow_nb_model.fit(X_train_vectorized, y_train)

bow_nb_y_pred = bow_nb_model.predict(X_test_vectorized)

bow_nb_accuracy = accuracy_score(y_test, bow_nb_y_pred) * 100

print(f"Accuracy: {bow_nb_accuracy}")
print(f"Classification Report:\n{classification_report(y_test, bow_nb_y_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, bow_nb_y_pred)}")

Accuracy: 98.20627802690582
Classification Report:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       0.97      0.89      0.93       149

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Confusion Matrix:
[[962   4]
 [ 16 133]]


In [83]:
## Logistic Regression

from sklearn.linear_model import LogisticRegression

bow_logr_model = LogisticRegression()
bow_logr_model.fit(X_train_vectorized, y_train)

bow_logr_y_pred = bow_logr_model.predict(X_test_vectorized)

bow_logr_accuracy = accuracy_score(y_test, bow_logr_y_pred) * 100

print(f"Accuracy: {bow_logr_accuracy}")
print(f"Classification Report:\n{classification_report(y_test, bow_logr_y_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, bow_logr_y_pred)}")

Accuracy: 98.11659192825111
Classification Report:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       966
        spam       1.00      0.86      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Confusion Matrix:
[[966   0]
 [ 21 128]]


In [16]:
## MLP Preparation (Can be used both BoW and TF-IDF)

print(f"y_train: {y_train.iloc[9]}")
y_train_encoded = le.fit_transform(y_train)
print(f"y_train encoded: {y_train_encoded[9]}\n")

print(f"y_test: {y_test.iloc[9]}")
y_test_encoded = le.fit_transform(y_test)
print(f"y_test encoded: {y_test_encoded[9]}")

y_train: spam
y_train encoded: 1

y_test: ham
y_test encoded: 0


In [17]:
## MLP dataSets, dataLoaders

import torch
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train_vectorized.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)

X_test_tensor = torch.tensor(X_test_vectorized.toarray(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

In [79]:
## MLP model

import torch.nn as nn

class MLP(nn.Module):
  def __init__(self, input_size, hidden_units, output_size):
    super().__init__()

    self.fc1 = nn.Sequential(
        nn.Linear(input_size, hidden_units),
        nn.ReLU()
    )

    self.fc2 = nn.Sequential(
        nn.Linear(hidden_units, output_size),
        nn.LogSoftmax(dim = 1)
    )

  def forward(self,x):
    return self.fc2(self.fc1(x))

mlp_model = MLP(input_size = X_train_vectorized.shape[1],
                hidden_units = 128,
                output_size = 2
                )

print(next(mlp_model.parameters()).device)
mlp_model

cpu


MLP(
  (fc1): Sequential(
    (0): Linear(in_features=8271, out_features=128, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=128, out_features=2, bias=True)
    (1): LogSoftmax(dim=1)
  )
)

In [78]:
## MLP loss and optimizer and accuracy function

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = mlp_model.parameters(),
                             lr = 0.001)

def accuracy_fn(y_true, y_pred):

    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [76]:
## MLP training loop (used for BoW and TF-IDF)

all_preds = []
all_labels = []
epochs = 10

for epoch in range(epochs):
    print(f"Epoch: {epoch}\n----------")
    mlp_model.train()
    train_loss = 0

    for batch, (X, y) in enumerate(train_loader):
        y_pred = mlp_model(X)
        loss = loss_fn(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # if batch % 50 == 0:
        #     print(f"Batch: {batch}, Loss: {loss.item():.4f}")

    train_loss /= len(train_loader)

    test_loss, test_acc = 0, 0
    mlp_model.eval()
    with torch.inference_mode():
        for X, y in test_loader:
            test_pred = mlp_model(X)
            test_loss += loss_fn(test_pred, y).item()
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
            _, preds = torch.max(test_pred, 1)
            all_preds.extend(preds.numpy())
            all_labels.extend(y.numpy())

    test_loss /= len(test_loader)
    test_acc /= len(test_loader)

    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f}%\n")

Epoch: 0
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 1
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 2
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 3
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 4
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 5
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 6
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 7
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 8
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%

Epoch: 9
----------

Train loss: 0.00000 | Test loss: 0.14651 | Test acc: 98.41%



In [77]:
## MLP statistics

print(f"Accuracy: {test_acc:2f}")
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds))

Accuracy: 98.405350
[[9590   70]
 [ 100 1390]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      9660
           1       0.95      0.93      0.94      1490

    accuracy                           0.98     11150
   macro avg       0.97      0.96      0.97     11150
weighted avg       0.98      0.98      0.98     11150



### TF-IDF

In [68]:
## TF-IDF preparation

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("Train TF-IDF shape:", X_train_tfidf.shape)
print("Test TF-IDF shape:", X_test_tfidf.shape)


print("Original text:", X_train.iloc[sample_index])
print("TF-IDF representation:", X_train_tfidf[sample_index].toarray())
print("Features:", tfidf_vectorizer.get_feature_names_out()[:20])

Train TF-IDF shape: (4457, 8271)
Test TF-IDF shape: (1115, 8271)
Original text: going nothing greatbye
TF-IDF representation: [[0. 0. 0. ... 0. 0. 0.]]
Features: ['008704050406' '0089my' '0121' '01223585236' '01223585334' '02' '020603'
 '0207' '02070836089' '02072069400' '02073162414' '02085076972' '020903'
 '021' '050703' '0578' '06' '060505' '061104' '07008009200']


In [69]:
## TF-IDF with Naive Bayes

from sklearn.naive_bayes import MultinomialNB

tfidf_model = MultinomialNB()
tfidf_model.fit(X_train_tfidf, y_train)

tfidf_nb_y_pred = tfidf_model.predict(X_test_tfidf)

tfidf_accuracy = accuracy_score(y_test, tfidf_nb_y_pred) * 100

print(f"Accuracy: {tfidf_accuracy}")
print(f"Classification Report:\n{classification_report(y_test, tfidf_nb_y_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, tfidf_nb_y_pred)}")

Accuracy: 96.05381165919282
Classification Report:
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       966
        spam       1.00      0.70      0.83       149

    accuracy                           0.96      1115
   macro avg       0.98      0.85      0.90      1115
weighted avg       0.96      0.96      0.96      1115

Confusion Matrix:
[[966   0]
 [ 44 105]]


In [70]:
## TF-IDF with Logistic Regression

from sklearn.linear_model import LogisticRegression

tfidf_logr_model = LogisticRegression(max_iter = 5000)
tfidf_logr_model.fit(X_train_tfidf, y_train)

tfidf_logr_y_pred = tfidf_logr_model.predict(X_test_tfidf)

tfidf_logr_accuracy = accuracy_score(y_test, tfidf_logr_y_pred) * 100

print(f"Accuracy: {tfidf_logr_accuracy}")
print(f"Classification Report:\n{classification_report(y_test, tfidf_logr_y_pred)}")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, tfidf_logr_y_pred)}")

Accuracy: 95.87443946188341
Classification Report:
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       966
        spam       0.99      0.70      0.82       149

    accuracy                           0.96      1115
   macro avg       0.97      0.85      0.90      1115
weighted avg       0.96      0.96      0.96      1115

Confusion Matrix: 
 [[965   1]
 [ 45 104]]


In [71]:
## MLP Preparation (Can be used both BoW and TF-IDF)

print(f"y_train: {y_train.iloc[9]}")
y_train_encoded = le.fit_transform(y_train)
print(f"y_train encoded: {y_train_encoded[9]}\n")

print(f"y_test: {y_test.iloc[9]}")
y_test_encoded = le.fit_transform(y_test)
print(f"y_test encoded: {y_test_encoded[9]}")

y_train: spam
y_train encoded: 1

y_test: ham
y_test encoded: 0


In [46]:
## MLP DataSet, DataLoader

import torch
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)

X_test_tensor = torch.tensor(X_test_tfidf.toarray(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

tfidf_train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
tfidf_test_loader = DataLoader(test_dataset, batch_size= 64, shuffle = False)

In [72]:
# ---- Training loop (csak loss/accuracy kiírással) ----
tfidf_epochs = 10

for tfidf_epoch in range(tfidf_epochs):
    mlp_model.train()
    tfidf_train_loss = 0

    for X, y in tfidf_train_loader:
        tfidf_y_pred = mlp_model(X)
        tfidf_loss = loss_fn(tfidf_y_pred, y)

        optimizer.zero_grad()
        tfidf_loss.backward()
        optimizer.step()
        tfidf_train_loss += tfidf_loss.item()

    tfidf_train_loss /= len(tfidf_train_loader)
    print(f"Epoch {tfidf_epoch+1}/{tfidf_epochs} - Train loss: {tfidf_train_loss:.7f}")

mlp_model.eval()
tfidf_mlp_final_preds, tfidf_mlp_final_labels = [], []

with torch.inference_mode():
    for X, y in tfidf_test_loader:
        out = mlp_model(X)
        _, preds = torch.max(out, 1)
        tfidf_mlp_final_preds.extend(preds.numpy())
        tfidf_mlp_final_labels.extend(y.numpy())

Epoch 1/10 - Train loss: 0.0000136
Epoch 2/10 - Train loss: 0.0000104
Epoch 3/10 - Train loss: 0.0000093
Epoch 4/10 - Train loss: 0.0000086
Epoch 5/10 - Train loss: 0.0000082
Epoch 6/10 - Train loss: 0.0000078
Epoch 7/10 - Train loss: 0.0000074
Epoch 8/10 - Train loss: 0.0000070
Epoch 9/10 - Train loss: 0.0000066
Epoch 10/10 - Train loss: 0.0000063


In [86]:
## MLP statistics

print("Accuracy:", accuracy_score(tfidf_mlp_final_labels, tfidf_mlp_final_preds))
print("Confusion Matrix:\n", confusion_matrix(tfidf_mlp_final_labels, tfidf_mlp_final_preds))
print("Classification Report:\n", classification_report(tfidf_mlp_final_labels, tfidf_mlp_final_preds))

Accuracy: 0.9838565022421525
Confusion Matrix:
 [[966   0]
 [ 18 131]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.88      0.94       149

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



### Summarise the statistics

In [99]:
## MLP statistics

import pandas as pd

results = []

nb_report = classification_report(y_test, bow_nb_y_pred, output_dict = True)
results.append(
    {
        'Model': 'BoW + Naive Bayes',
        'Accuracy' : round(nb_report['accuracy'] * 100, 2),
        'Precision (spam)': round(nb_report['spam']['precision'] * 100, 2),
        'Recall (spam)': round(nb_report['spam']['recall'] * 100,2),
        'F1-score (spam)': round(nb_report['spam']['f1-score'] * 100,2),
        'Precision (ham)': round(nb_report['ham']['precision'] * 100,2),
        'Recall (ham)': round(nb_report['ham']['recall'] * 100,2),
        'F1-score (ham)': round(nb_report['ham']['f1-score'] * 100,2)
    }
)

logr_report = classification_report(y_test, bow_logr_y_pred, output_dict = True)
results.append(
    {
        'Model': 'BoW + Logistic Regression',
        'Accuracy' : round(logr_report['accuracy'] * 100, 2),
        'Precision (spam)': round(logr_report['spam']['precision'] * 100, 2),
        'Recall (spam)': round(logr_report['spam']['recall'] * 100, 2),
        'F1-score (spam)': round(logr_report['spam']['f1-score'] * 100, 2),
        'Precision (ham)' : round(logr_report['ham']['precision'] * 100, 2),
        'Recall (ham)' : round(logr_report['ham']['recall'] * 100, 2),
        'F1-score (ham)' : round(logr_report['ham']['f1-score'] * 100, 2)
    }
)

mlp_report = classification_report(all_labels, all_preds, output_dict = True)
results.append(
    {
        'Model': 'Bow + MLP',
        'Accuracy' : round(mlp_report['accuracy'] * 100, 2),
        'Precision (spam)': round(mlp_report['1']['precision']*100,2),
        'Recall (spam)': round(mlp_report['1']['recall']*100,2),
        'F1-score (spam)': round(mlp_report['1']['f1-score']*100,2),
        'Precision (ham)': round(mlp_report['0']['precision']*100,2),
        'Recall (ham)': round(mlp_report['0']['recall']*100,2),
        'F1-score (ham)': round(mlp_report['0']['f1-score']*100,2)
    }
)

tfidf_report = classification_report(y_test, tfidf_nb_y_pred, output_dict = True)
results.append(
    {
        'Model': 'TF-IDF + Naive Bayes',
        'Accuracy' : round(tfidf_report['accuracy'] * 100,2),
        'Precision (spam)': round(tfidf_report['spam']['precision'] * 100,2),
        'Recall (spam)': round(tfidf_report['spam']['recall'] * 100,2),
        'F1-score (spam)': round(tfidf_report['spam']['f1-score'] * 100,2),
        'Precision (ham)': round(tfidf_report['ham']['precision'] * 100,2),
        'Recall (ham)': round(tfidf_report['ham']['recall'] * 100,2),
        'F1-score (ham)': round(tfidf_report['ham']['f1-score'] * 100,2)
    }
)

tfidf_logr_report = classification_report(y_test, tfidf_logr_y_pred, output_dict = True)
results.append(
    {
        'Model': 'TF-IDF + Logistic Regression',
        'Accuracy' : round(tfidf_logr_report['accuracy'] * 100,2),
        'Precision (spam)': round(tfidf_logr_report['spam']['precision'] * 100,2),
        'Recall (spam)': round(tfidf_logr_report['spam']['recall'] * 100,2),
        'F1-score (spam)': round(tfidf_logr_report['spam']['f1-score'] * 100,2),
        'Precision (ham)': round(tfidf_logr_report['ham']['precision'] * 100,2),
        'Recall (ham)': round(tfidf_logr_report['ham']['recall'] * 100,2),
        'F1-score (ham)': round(tfidf_logr_report['ham']['f1-score'] * 100,2)
    }
)

tfidf_mlp_report = classification_report(tfidf_mlp_final_labels, tfidf_mlp_final_preds, output_dict = True)
results.append(
    {
        'Model': 'TF-IDF + MLP',
        'Accuracy' : round(tfidf_mlp_report['accuracy'] * 100,2),
        'Precision (spam)': round(tfidf_mlp_report['1']['precision']*100,2),
        'Recall (spam)': round(tfidf_mlp_report['1']['recall']*100,2),
        'F1-score (spam)': round(tfidf_mlp_report['1']['f1-score']*100,2),
        'Precision (ham)': round(tfidf_mlp_report['0']['precision']*100,2),
        'Recall (ham)': round(tfidf_mlp_report['0']['recall']*100,2),
        'F1-score (ham)': round(tfidf_mlp_report['0']['f1-score']*100,2)
    }
)

df_results = pd.DataFrame(results)
df_results

,Model,Accuracy,Precision (spam),Recall (spam),F1-score (spam),Precision (ham),Recall (ham),F1-score (ham)
0,BoW + Naive Bayes,98.21,97.08,89.26,93.01,98.36,99.59,98.97
1,BoW + Logistic Regression,98.12,100.00,85.91,92.42,97.87,100.00,98.92
2,Bow + MLP,98.48,95.21,93.29,94.24,98.97,99.28,99.12
3,TF-IDF + Naive Bayes,96.05,100.00,70.47,82.68,95.64,100.00,97.77
4,TF-IDF + Logistic Regression,95.87,99.05,69.80,81.89,95.54,99.90,97.67
5,TF-IDF + MLP,98.39,100.00,87.92,93.57,98.17,100.00,99.08
